In [14]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [15]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

PART 10 - LOGICAL AND SEMANTIC ROUTING

In [16]:
from typing import Literal
from pydantic import BaseModel, Field  # Updated import
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Data model for routing queries
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""
    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question, choose which datasource would be most relevant for answering their question",
    )

# Initialize the LLM with function call
llm = ChatGroq(temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Define the system prompt
system = """You are an expert at routing a user question to the appropriate data source.
Based on the programming language the question is referring to, route it to the relevant data source."""

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define the router
router = prompt | structured_llm

# Example usage
def route_query(question: str) -> RouteQuery:
    try:
        # Use the router with the correct input format
        result = router.invoke({"question": question})
        return result
    except Exception as e:
        print(f"Error routing query: {e}")
        return None

# Example query
result = route_query("How do I create a list in Python?")
print(result)


datasource='python_docs'


In [17]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [18]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [19]:
full_chain.invoke({"question": question})

'chain for python_docs'

Semantic Routing

In [21]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
prompt_templates = [physics_template, math_template]
prompt_embeddings = hf_embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = hf_embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGroq()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'model_kwargs'

QUERY CONSTRUCTION

PART 11 - QUERY STRUCTURING FOR METADATA FILTERS

In [22]:
import yt_dlp
import os
from datetime import datetime

# Disable parallelism warning from Hugging Face
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def get_youtube_metadata(url: str):
    try:
        ydl_opts = {
            'quiet': True,
            'noplaylist': True,
            'extract_flat': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
        
        # Format the date if available
        raw_date = info.get("upload_date", "Unknown Date")
        formatted_date = (
            datetime.strptime(raw_date, "%Y%m%d").strftime("%Y-%m-%d")
            if raw_date.isdigit() else raw_date
        )
        
        metadata = {
            "Video Title": info.get("title", "Unknown Title"),
            "Video URL": info.get("webpage_url", "Unknown URL"),
            "Publish Date": formatted_date,
            "Author": info.get("uploader", "Unknown Author"),
        }
        
        return metadata

    except Exception as e:
        print(f"Error extracting YouTube metadata: {e}")
        return None

# Example usage
video_url = "https://www.youtube.com/watch?v=u3c8OQaeLWo"
metadata = get_youtube_metadata(video_url)

# Print each metadata field on a new line
if metadata:
    print("\n".join(f"{key}: {value}" for key, value in metadata.items()))

Video Title: Build ANYTHING With AI Agents For FREE! (DeepSeek-R1 Beats ChatGPT)
Video URL: https://www.youtube.com/watch?v=u3c8OQaeLWo
Publish Date: 2025-02-01
Author: Ishan Sharma


In [23]:
import datetime
from typing import Optional
from pydantic import BaseModel, Field

class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.model_fields:
            value = getattr(self, field)
            if value is not None:
                print(f"{field}: {value}")

# Example usage
search_query = TutorialSearch(
    content_search="machine learning basics",
    title_search="ML tutorial",
    min_view_count=1000,
    earliest_publish_date=datetime.date(2023, 1, 1),
)
search_query.pretty_print()

content_search: machine learning basics
title_search: ML tutorial
min_view_count: 1000
earliest_publish_date: 2023-01-01


Now, we prompt the LLM to produce queries

In [24]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

# System prompt defining the LLM's behavior
system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""

# Creating a prompt template that formats user input for the LLM
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),  # Defines the role and behavior of the model
        ("human", "{question}"),  # Placeholder for user-provided questions
    ]
)

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(api_key = groq_api_key, model_name="llama3-8b-8192")
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [25]:
query_analyzer.invoke({"question": "rag from scratch"}).pretty_print()

content_search: rebuild from scratch
title_search: rebuild from scratch


In [26]:
query_analyzer.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

content_search: chat langchain published 2023
title_search: chat langchain published 2023
min_view_count: 0
max_view_count: 1000000
earliest_publish_date: 2023-01-01
latest_publish_date: 2023-12-31
min_length_sec: 0
max_length_sec: 1000000


In [27]:
query_analyzer.invoke(
    {"question": "videos that are focused on the topic of chat langchain that are published before 2024"}
).pretty_print()

content_search: chat langchain
title_search: chat langchain
earliest_publish_date: 2023-12-31


In [28]:
query_analyzer.invoke(
    {"question": "how to use multi-modal models in an agent, only videos under 5 minutes"}
).pretty_print()

content_search: multi-modal models in an agent
title_search: 
min_length_sec: 0
max_length_sec: 300
